In [1]:
import fitz  # PyMuPDF
import os
import re


def clean_text(text):
    """
    Membersihkan dan merapikan teks hasil ekstraksi.
    """
    # Menghapus multiple whitespace
    text = re.sub(r'\s+', ' ', text)
    # Menghapus hyphenation di akhir baris
    text = re.sub(r'(\w+)-\s*\n\s*(\w+)', r'\1\2', text)
    # Menghapus line breaks yang tidak diperlukan
    text = re.sub(r'\n+', ' ', text)
    return text.strip()


def load_pdf_text(pdf_path):
    """
    Load dan ekstrak teks dari PDF.
    """
    try:
        doc = fitz.open(pdf_path)
        full_text = ""
        
        # Ekstrak teks per halaman dengan pembersihan
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            page_text = page.get_text("text")
            # Bersihkan teks per halaman
            page_text = clean_text(page_text)
            full_text += page_text + " "
        
        return clean_text(full_text)
    except Exception as e:
        print(f"Terjadi kesalahan: {e}")
        return ""


def recursive_chunk(text, chunk_size, separators=None):
    """
    Membagi teks secara rekursif menjadi chunk dengan panjang maksimum menggunakan separator yang ditentukan.
    """
    if not text.strip():  # Jika teks kosong, hentikan rekursi
        return []

    if separators is None:
        separators = ["."]

    for separator in separators:
        if separator in text[:chunk_size]:
            split_index = text[:chunk_size].rfind(separator) + len(separator)
            chunk = text[:split_index]
            remaining_text = text[split_index:].strip()
            return [chunk] + recursive_chunk(remaining_text, chunk_size, separators)

    # Jika tidak ada separator yang ditemukan dalam batas chunk_size
    chunk = text[:chunk_size]
    remaining_text = text[chunk_size:].strip()
    return [chunk] + recursive_chunk(remaining_text, chunk_size, separators)


def save_chunks_to_single_file(chunks, output_file):
    """
    Menyimpan semua chunks ke dalam satu file dengan pemisah yang jelas.
    """
    # Pastikan direktori output ada
    output_dir = os.path.dirname(output_file)
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)

    with open(output_file, "w", encoding="utf-8") as f:
        for idx, chunk in enumerate(chunks, 1):
            # Tulis header chunk
            f.write(f"\n{'='*50}\n")
            f.write(f"CHUNK {idx:03d}\n")
            f.write(f"{'='*50}\n\n")
            
            # Tulis konten chunk
            f.write(chunk)
            f.write("\n")  # Tambah baris kosong di akhir chunk

        # Tulis ringkasan di akhir file
        f.write(f"\n{'='*50}\n")
        f.write(f"RINGKASAN\n")
        f.write(f"{'='*50}\n")
        f.write(f"Total Chunks: {len(chunks)}\n")
        
        # Tambah statistik per chunk
        for idx, chunk in enumerate(chunks, 1):
            words = len(chunk.split())
            sentences = len(re.split(r'(?<=[.!?])\s+', chunk.strip()))
            f.write(f"\nChunk {idx:03d}:\n")
            f.write(f"- Jumlah kata: {words}\n")
            f.write(f"- Jumlah kalimat: {sentences}\n")


def main():
    # Konfigurasi
    pdf_path = "C:/Users/User/pdf_chunking_project/Dokumen.pdf"
    output_file = "output_chunks.txt"  # File output tunggal
    chunk_size = 1000

    # Load PDF dan ekstrak teks
    print("Mengekstrak teks dari PDF...")
    full_text = load_pdf_text(pdf_path)

    if not full_text:
        print("Tidak ada teks yang bisa diekstrak. Program dihentikan.")
        return

    # Chunk teks secara rekursif
    print("Membagi teks menjadi chunks secara rekursif...")
    chunks = recursive_chunk(full_text, chunk_size)

    print(f"Selesai membagi menjadi {len(chunks)} chunks.")

    # Simpan semua chunks ke satu file
    save_chunks_to_single_file(chunks, output_file)
    print(f"Semua chunks telah disimpan ke file: {output_file}")
    
    # Tampilkan preview struktur file
    print("\nStruktur file output:")
    print("1. Setiap chunk dipisahkan dengan garis pembatas '='")
    print("2. Setiap chunk memiliki header dengan nomor chunk")
    print("3. Di akhir file terdapat ringkasan statistik untuk semua chunks")


if __name__ == "__main__":
    main()

Mengekstrak teks dari PDF...
Membagi teks menjadi chunks secara rekursif...
Selesai membagi menjadi 5 chunks.
Semua chunks telah disimpan ke file: output_chunks.txt

Struktur file output:
1. Setiap chunk dipisahkan dengan garis pembatas '='
2. Setiap chunk memiliki header dengan nomor chunk
3. Di akhir file terdapat ringkasan statistik untuk semua chunks


In [2]:
import fitz  # PyMuPDF
import os
import re
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

def clean_text(text):
    """
    Membersihkan dan merapikan teks hasil ekstraksi.
    """
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'(\w+)-\s*\n\s*(\w+)', r'\1\2', text)
    text = re.sub(r'\n+', ' ', text)
    return text.strip()

def load_pdf_text(pdf_path):
    """
    Load dan ekstrak teks dari PDF.
    """
    try:
        doc = fitz.open(pdf_path)
        full_text = ""
        
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            page_text = page.get_text("text")
            page_text = clean_text(page_text)
            full_text += page_text + " "
        
        return clean_text(full_text)
    except Exception as e:
        print(f"Terjadi kesalahan: {e}")
        return ""

def recursive_chunk(text, chunk_size, separators=None):
    """
    Membagi teks secara rekursif menjadi chunk dengan panjang maksimum menggunakan separator yang ditentukan.
    """
    if not text.strip():
        return []

    if separators is None:
        separators = ["."]

    for separator in separators:
        if separator in text[:chunk_size]:
            split_index = text[:chunk_size].rfind(separator) + len(separator)
            chunk = text[:split_index]
            remaining_text = text[split_index:].strip()
            return [chunk] + recursive_chunk(remaining_text, chunk_size, separators)

    chunk = text[:chunk_size]
    remaining_text = text[chunk_size:].strip()
    return [chunk] + recursive_chunk(remaining_text, chunk_size, separators)

def save_chunks_to_files(chunks, output_folder):
    """
    Menyimpan setiap chunk ke file terpisah.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    chunk_files = []
    for idx, chunk in enumerate(chunks, 1):
        chunk_file = os.path.join(output_folder, f"chunk_{idx:03d}.txt")
        with open(chunk_file, "w", encoding="utf-8") as f:
            f.write(chunk)
        chunk_files.append(chunk_file)
    
    return chunk_files

def summarize_text(text):
    """
    Summarize text using LLM.
    """
    OLLAMA_MODEL = "llama3.2"
    llm = OllamaLLM(model=OLLAMA_MODEL)
    
    template = """
    Anda adalah asisten AI yang ahli dalam menganalisis dokumen.  
    Berdasarkan dokumen berikut, identifikasi perubahan nama, restrukturisasi, dan strategi ekspansi PT Pertamina.  
    Gunakan hanya informasi yang terdapat dalam dokumen.  

    Dokumen:
    "{document}"
    Ringkasan:
    """
    
    prompt = ChatPromptTemplate.from_template(template)
    chain = prompt | llm
    response = chain.invoke({"document": text})
    return response

def main():
    # Konfigurasi
    pdf_path = "C:/Users/User/pdf_chunking_project/Dokumen.pdf"
    output_folder = "output_chunks"  # Folder untuk menyimpan chunks
    chunk_size = 1000

    # Load PDF dan ekstrak teks
    print("Mengekstrak teks dari PDF...")
    full_text = load_pdf_text(pdf_path)

    if not full_text:
        print("Tidak ada teks yang bisa diekstrak. Program dihentikan.")
        return

    # Chunk teks secara rekursif
    print("Membagi teks menjadi chunks secara rekursif...")
    chunks = recursive_chunk(full_text, chunk_size)

    print(f"Selesai membagi menjadi {len(chunks)} chunks.")

    # Simpan setiap chunk ke file terpisah
    chunk_files = save_chunks_to_files(chunks, output_folder)
    print(f"Semua chunks telah disimpan ke folder: {output_folder}")

    # Summarize setiap chunk
    summaries = []
    for chunk_file in chunk_files:
        with open(chunk_file, "r", encoding="utf-8") as f:
            chunk_text = f.read()
        
        summary = summarize_text(chunk_text)
        summaries.append(summary)
        print(f"Ringkasan untuk {chunk_file}:\n{summary}\n")

    # Simpan semua ringkasan ke satu file
    summary_file = os.path.join(output_folder, "summary.txt")
    with open(summary_file, "w", encoding="utf-8") as f:
        for idx, summary in enumerate(summaries, 1):
            f.write(f"Ringkasan Chunk {idx:03d}:\n")
            f.write(summary)
            f.write("\n\n")
    
    print(f"Semua ringkasan telah disimpan ke file: {summary_file}")

if __name__ == "__main__":
    main()

Mengekstrak teks dari PDF...
Membagi teks menjadi chunks secara rekursif...
Selesai membagi menjadi 5 chunks.
Semua chunks telah disimpan ke folder: output_chunks
Ringkasan untuk output_chunks\chunk_001.txt:
Berdasarkan dokumen yang diberikan, dapat diidentifikasi beberapa perubahan nama dan restrukturisasi PT Pertamina sebagai berikut:

Perubahan Nama:

1. PT Eksploitasi Tambang Minyak Sumatera Utara (1950-an) menjadi PT Perusahaan Minyak Nasional (PERMINA) pada 10 Desember 1957.
2. PERMINA ditetapkan menjadi Perusahaan Negara (PN) dengan nama PN Pertambangan Minyak Nasional (Permina) pada 1 Juli 1961.
3. PN Permina bergabung dengan PN Pertamin pada 20 Agustus 1968, sehingga menjadi sebuah perusahaan baru bernama PN Pertambangan Minyak dan Gas Bumi Negara (Pertamina).

Restrukturisasi:

1. Pada 1950-an, PT Eksploitasi Tambang Minyak Sumatera Utara didirikan untuk mengelola ladang minyak di wilayah Sumatera.
2. Setelah perubahan nama menjadi PERMINA pada 10 Desember 1957, perusahaan te